In [2]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [14]:
from __future__ import print_function
import time
import numpy as np
import argparse
import io
import sys
import datetime
import ucert
import os

In [2]:
diff = False
infiles = "/Users/TRD/Research_Personal/Quant_Mo/videos/sample_16/"
output = "/Users/TRD/Research_Personal/Quant_Mo/sample_16_mem"


In [18]:
os.listdir(infiles)

['aw_motion!4@2018-08-16.12:39:22.npy',
 'aw_motion!4@2018-08-16.15:45:38.npy',
 'aw_motion!4@2018-08-16.15:05:18.npy',
 'aw_motion!4@2018-08-16.14:40:42.npy',
 'aw_motion!4@2018-08-16.13:24:07.npy',
 'aw_motion!4@2018-08-16.12:51:38.npy',
 'aw_motion!4@2018-08-16.12:35:19.npy',
 'aw_motion!4@2018-08-16.12:16:05.npy',
 'aw_motion!4@2018-08-16.15:33:34.npy',
 'aw_motion!4@2018-08-16.12:08:10.npy',
 'aw_motion!4@2018-08-16.13:36:16.npy',
 'aw_motion!4@2018-08-16.12:16:04.npy',
 'aw_motion!4@2018-08-16.13:12:03.npy',
 'aw_motion!4@2018-08-16.15:17:20.npy',
 'aw_motion!4@2018-08-16.13:03:44.npy',
 'aw_motion!4@2018-08-16.13:15:52.npy',
 'aw_motion!4@2018-08-16.14:48:53.npy',
 'aw_motion!4@2018-08-16.15:29:34.npy',
 'aw_motion!4@2018-08-16.12:59:38.npy',
 'aw_motion!4@2018-08-16.13:20:03.npy',
 'aw_motion!4@2018-08-16.14:44:46.npy',
 'aw_motion!4@2018-08-16.14:57:06.npy',
 'aw_motion!4@2018-08-16.12:31:27.npy',
 'aw_motion!4@2018-08-16.15:25:23.npy',
 'aw_motion!4@2018-08-16.13:44:25.npy',


In [3]:
def process(file):
    try:
        numObj = np.load(file)
        [date, arr] = numObj
        return [date, arr]
    except EOFError:
        return []

In [4]:
def sizeof_fmt(num, suffix='B'):
    # Took this from online to read how much RAM this is using
    for unit in ['', 'Ki', 'Mi', 'Gi', 'Ti', 'Pi', 'Ei', 'Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [11]:
def absoluteFilePaths(directory):
    filenames = os.listdir(directory)
    return [directory+i for i in filenames if '.npy' in i ]

In [6]:
def saveArr(x):
    # Test to verify we're passing in the correct motion vectors
    assert x.shape[2] == 2
    t = datetime.datetime.now().timestamp()
    arrayStorage = np.array([t, x.copy()], dtype=object)
    # print('Saving an array of size: {}'.format(sizeof_fmt(sys.getsizeof(arrayStorage))))
    np.save(output, arrayStorage)

In [7]:
def saveTxt(u_array):
    with open(output+'.txt','w+') as outfile:
        for i in u_array:
            np.savetxt(outfile, i, fmt='%r')

In [8]:
def reshapeHelp(arr):
    # This function converts the array into an added dimension to support concatenation later
    assert type(arr).__module__ == np.__name__
    lis = list(arr.shape)
    lis.insert(0, 1)
    return np.reshape(arr, tuple(lis))

In [12]:
diff = False
files = absoluteFilePaths(infiles)
assert files != None # To validate that we don't have a null array
print("Num of files:\t{}".format(len(files)))
dates = []
tempArr = np.array([], dtype=int)

NameError: name 'os' is not defined

In [ ]:
def combine(files):
    if diff:
        # This will need to be modified in the future
        assert len(files) >= 2
        [a_date, a_arr] = np.load(files[0])
        [b_date, b_arr] = np.load(files[1])
        u_array = b_arr - a_arr
    else:
        for i in files:
            temp = process(i)
            if temp:
                [temp_date, arr] = temp
                dates.append(temp_date)
                if sys.getsizeof(tempArr) == 96:
                    tempArr = reshapeHelp(arr)
                else:
                    tempIn = reshapeHelp(arr)
                    tempArr = np.concatenate((tempArr, tempIn), axis=0)
    A = np.mean(tempArr, axis=0)
    return A

In [20]:
diff = False
files = absoluteFilePaths(infiles)
assert files != None # To validate that we don't have a null array
print("Num of files:\t{}".format(len(files)))
dates = []
tempArr = np.array([], dtype=int)

print("Output is:\t{}".format(output))

if diff:
    # This will need to be modified in the future
    assert len(files) >= 2
    [a_date, a_arr] = np.load(files[0])
    [b_date, b_arr] = np.load(files[1])
    u_array = b_arr - a_arr
else:
    for i in files:
        temp = process(i)
        if temp:
            [temp_date, arr] = temp
            dates.append(temp_date)
            if sys.getsizeof(tempArr) == 96:
                tempArr = reshapeHelp(arr)
            else:
                tempIn = reshapeHelp(arr)
                tempArr = np.concatenate((tempArr, tempIn), axis=0)

            #print('Shape of the array is: {}'.format(tempArr.shape))
            #print('Size of the array is: {}\n'.format(sizeof_fmt(sys.getsizeof(tempArr))))

    u_array = np.mean(tempArr, axis=0)

print('Size of the final average: {}\n'.format(sizeof_fmt(sys.getsizeof(u_array))))
print(u_array.shape)

saveTxt(u_array)

Num of files:	65
Output is:	/Users/TRD/Research_Personal/Quant_Mo/sample_16_mem
Size of the final average: 64.1KiB

(64, 64, 2)
